![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir si el precio del automóvil es alto o no. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [24]:
import warnings
warnings.filterwarnings('ignore')
#algo

In [25]:
# Importación de librerías
%matplotlib inline
import pandas as pd
import numpy as np

# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data['HighPrice'] = (data['Price'] > data['Price'].mean()).astype(int)
data = data.drop(['Model', 'Price'], axis=1)

# Visualización dataset
data.head()

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE,HighPrice
7,2014,6480,0,0,0,1,0,0,0,1
11,2014,39972,0,0,0,0,1,0,0,0
167,2016,18989,0,0,0,0,0,1,0,1
225,2014,51330,0,0,0,1,0,0,0,0
270,2007,116065,0,1,0,0,0,0,0,0


In [26]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['HighPrice']
X = data.drop(['HighPrice'], axis=1)

In [27]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el acurracy del modelo en el set de test.

In [29]:
# Celda 4

# Definición de la función que calcula el error cuadrático de las particiones (con respecto al promedio)
def MSE_particion(y_l, y_r):
    
    return (sum([(x-y_l.mean())**2 for x in y_l])+sum([(x-y_r.mean())**2 for x in y_r]))/(len(y_l)+len(y_r))
    
def mejor_particion(X, y): 
    
    mejor_particion = [0,0,np.inf]  # j, split, mse
    
    for j in X.columns:
        
        splits = X.loc[:,j].unique()
        for split in splits:
            mse = MSE_particion(y[X[j] < split], y[X[j] >= split])
            
            if mse < mejor_particion[2]:
                mejor_particion = [j, split, mse]
    
    if (len(np.unique(X.loc[X[mejor_particion[0]] < mejor_particion[1],mejor_particion[0]])) == 0) or (len(np.unique(X.loc[X[mejor_particion[0]] >= mejor_particion[1],mejor_particion[0]])) == 0):
        mejor_particion[1] = -1
    
    return(mejor_particion)

def tree_grow(X, y, level=0):
    
    if X.shape[0] == 1:
        tree = dict(y_pred=y.iloc[:1].values[0], level=level, split=-1, n_samples=1, MSE=0)
        return(tree)
    
    j, split, mse = mejor_particion(X, y)
    
    tree = dict(y_pred=y.mean(), level=level, split=-1, n_samples=X.shape[0], MSE=mse)
    
    if mse == np.inf:
        return tree
    if (len(np.unique(X[X[j] < split][j])) <= 1) or (len(np.unique(X[X[j] >= split][j])) <= 1):
        return(tree)
    
    
    # Continuar creando la partición
    X_l, y_l = X[X[j] < split], y[X[j] < split]
    X_r, y_r = X[X[j] >= split], y[X[j] >= split]
    tree['split'] = [j, split]

    # Siguiente iteración para cada partición
    
    tree['sl'] = tree_grow(X_l, y_l, level + 1)
    tree['sr'] = tree_grow(X_r, y_r, level + 1)
    
    return tree

def tree_predict(X, tree):
    
    predicted = np.ones(X.shape[0])
    
    # Revisar si es el nodo final
    if tree['split'] == -1:
        predicted = predicted * tree['y_pred']
            
    else:
        
        j, split = tree['split']
        filter_l = (X.loc[:, j] < split)
        X_l = X.loc[filter_l]
        X_r = X.loc[~filter_l]

        predicted[filter_l] = tree_predict(X_l, tree['sl'])
        predicted[~filter_l] = tree_predict(X_r, tree['sr'])

    return predicted



In [ ]:
# Celda 1
tree = tree_grow(X_train, y_train, level=0)

### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de clasificación y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 2


### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de clasificación y el parámetro `max_features` igual a `log(n_features)`. Presenten el acurracy del modelo en el set de test y comenten sus resultados.

In [ ]:
# Celda 3


### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para clasificación y presenten el acurracy del modelo en el set de test y comenten sus resultados.

In [ ]:
# Celda 4


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para clasificación. Presenten el acurracy del modelo en el set de test, comenten sus resultados y análicen cómo cada parámetro afecta el modelo.

In [ ]:
# Celda 5


### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de clasificación con la librería sklearn, presenten el acurracy del modelo en el set de test y comenten sus resultados.

In [ ]:
# Celda 6


### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para clasificación. Presenten el acurracy del modelo en el set de test, comenten sus resultados y análicen cómo cada parámetro afecta el modelo.

In [ ]:
# Celda 7


### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [ ]:
# Celda 8
